In [1]:
#https://en.wikipedia.org/wiki/Pythagorean_expectation#Use_in_pro_football

import pandas as pd
import numpy as np
import os
import glob

pd.options.mode.chained_assignment = None 
#clean up numerical results that wasn’t easily fixed by using the round() method
from decimal import Decimal

In [5]:
# location of pbp data
#source_folder = '/Users/justynrodrigues/Documents/nfl/data/pbp/csv/'
#all_files = glob.glob('/Users/justynrodrigues/Documents/nfl/data/pbp/csv/*.csv.gz')
#df = pd.concat((pd.read_csv(__, low_memory=False, index_col=0) for __ in all_files))

In [12]:
#df = pd.read_csv('/Users/justynrodrigues/Documents/nfl/data/pbp/csv/play_by_play_2021.csv.gz', low_memory=False, index_col=0)
df = pd.read_csv('https://raw.githubusercontent.com/nflverse/nfldata/master/data/games.csv')
df = df.query('season == 2021')
df = df[df.home_score.notnull()]
df['home_result'] = df['home_score'] - df['away_score']

In [9]:
#print([col for col in df.columns if 'home' in col])

In this first code block in the loop, we want to get all the points for, the points against, and the number of wins at home for each team. To do that we perform the following steps:

1. Filter the dataframe to include data for each team when they play a home game
2. Create a new column called “win” that takes information from the home_result column and gives us a “1” if the team won at home or a “0” if the team lost at home. This can be done using np.where() which is a very nice feature of numpy to use to segment your data. You can see from the original dataframe that a negative number in the home_result column denotes a loss.
3. Get a count of the total number of home games played so far in the dataframe (could essentially use the len() method as well).
4. Calculate the sum of the win column to get total number of wins at home
5. Calculate the total points for scored at home
6. Calculate the total points against scored at home

In [13]:
# First we will create a list of the 32 teams, and then initialize several empty lists that will be populated throughout the loop for each team.
teams = list(set(df.home_team.values))
wins = []
py_wins = []
total_points_for_list = []
total_points_against_list = []
 
# Now we will iterate over each team in the dataframe and begin our calculations: 
for team in teams:
 
    data_home = df[df.home_team == team]
    data_home['win'] = np.where(data_home.home_result > 0, 1, 0)
    home_count = data_home.home_team.count()
    win_home = data_home.win.sum()
    points_for_home = data_home.home_score.sum()
    points_against_home = data_home.away_score.sum()
    # same calculations for when the team plays away from home
    data_away = df[df.away_team == team]
    data_away['win'] = np.where(data_away.home_result < 0, 1, 0)
    away_count = data_away.away_team.count()
    win_away = data_away.win.sum()
    points_for_away = data_away.away_score.sum()
    points_against_away = data_away.home_score.sum()
    # Now we can finish out our calculations in the loop for each team and start populating the empty lists we initialized right before we started our loop.
    total_points_for = int(points_for_home + points_for_away)
    total_points_for_list.append(total_points_for)
     
    total_points_against = int(points_against_home + points_against_away)
    total_points_against_list.append(total_points_against)
    # In this code block we take the home points for and away points for to get the total points for scored by the team and populate that list. We do the same for total points against. 
    total_games = home_count + away_count
    total_games_left = 17 - total_games
    total_wins = win_home + win_away
    wins.append(total_wins)
     
    # Dynamic Exponent from Football Outsiders.  Static exponent would be 2.37 from Wiki
    # https://www.footballoutsiders.com/dvoa-ratings/2011/week-13-dvoa-ratings
    # In this code block, we can calculate the total games played by each team so far. With a 16 game season, we can then calculate the total games left.
    # # Then we get the total wins from the count of home wins and away wins we calculated earlier.
    # can also use 2.37
    exponent = 1.5 * np.log10((total_points_for + total_points_against)/total_games)
 
    pythagorean_wins = round(Decimal(total_games_left*(total_points_for**exponent)/((total_points_for**exponent) + (total_points_against**exponent))), 2)
    py_wins.append(pythagorean_wins)

In [14]:
# Now we just want to create a dataframe with our newly populated lists to eventually make a nice styled dataframe.

projected_wins = pd.DataFrame(list(zip(teams, total_points_for_list, total_points_against_list, py_wins, wins)), columns =[
    'Team', 'Points For', 'Points Against', 'Projected_Wins', 'Current_Wins'])
projected_wins['Total_Projected_Wins'] = projected_wins.Projected_Wins + projected_wins.Current_Wins
projected_wins.head()

,Team,Points For,Points Against,Projected_Wins,Current_Wins,Total_Projected_Wins
0,BUF,566,348,-1.55,12,10.45
1,MIN,425,426,0.00,8,8.00
2,SEA,395,366,0.00,7,7.00
3,NYJ,310,504,0.00,4,4.00
4,LAC,474,459,0.00,9,9.00


In [15]:
projected_wins['Win_Diff'] = projected_wins.Total_Projected_Wins - projected_wins.Current_Wins

In [16]:
afc = ['PIT', 'KC', 'BAL', 'BUF', 'TEN', 'MIA', 'IND', 'LV', 'CLE', 'NE', 'LAC', 'CIN', 'DEN', 'HOU', 'JAX', 'NYJ']
nfc = ['NO', 'GB', 'SEA', 'LA', 'ARI', 'TB', 'CHI', 'SF', 'MIN', 'PHI', 'DET', 'CAR', 'ATL', 'WAS', 'NYG', 'DAL']
afc_wins = projected_wins[projected_wins.Team.isin(afc)].sort_values('Total_Projected_Wins', ascending = False).reset_index(drop = True)
nfc_wins = projected_wins[projected_wins.Team.isin(nfc)].sort_values('Total_Projected_Wins', ascending = False).reset_index(drop = True)

There is a lot going on here:

1. Create two different lists for AFC and NFC teams
2. Create two separate dataframes for each conference first by filtering on which teams are in, using isin(), each conference list
3. Then we sort the new dataframes by Total Projected Wins to give us the teams with the highest amount of projected wins listed first in our dataframe and in descending order.
4. Then reset the index.

In [17]:
afc_wins = afc_wins[['Team', 'Total_Projected_Wins', 'Current_Wins', 'Win_Diff']]
afc_wins.columns = ['Team', 'Projected Wins', 'Actual Wins', 'Win Difference']
 
nfc_wins = nfc_wins[['Team', 'Total_Projected_Wins', 'Current_Wins', 'Win_Diff']]
nfc_wins.columns = ['Team', 'Projected Wins', 'Actual Wins', 'Win Difference']

COLORS = {'ARI':'#97233F','ATL':'#A71930','BAL':'#241773','BUF':'#00338D','CAR':'#0085CA','CHI':'#00143F',
          'CIN':'#FB4F14','CLE':'#FB4F14','DAL':'#B0B7BC','DEN':'#002244','DET':'#046EB4','GB':'#24423C',
          'HOU':'#C9243F','IND':'#003D79','JAX':'#136677','KC':'#CA2430','LA':'#002147','LAC':'#2072BA',
          'MIA':'#0091A0','MIN':'#4F2E84','NE':'#0A2342','NO':'#A08A58','NYG':'#192E6C','NYJ':'#203731',
          'LV':'#C4C9CC','PHI':'#014A53','PIT':'#FFC20E','SEA':'#7AC142','SF':'#C9243F','TB':'#D40909',
          'TEN':'#4095D1','WAS':'#FFC20F'}

def highlight_cols(s, coldict, conf = 'AFC'):
    if conf == 'NFC':
        return ['background-color: {}'.format(COLORS[v]) if v else '' for v in nfc_wins.Team.isin(COLORS.keys())*nfc_wins.Team.values]
    return ['background-color: {}'.format(COLORS[v]) if v else '' for v in afc_wins.Team.isin(COLORS.keys())*afc_wins.Team.values]

Again, lots going on here:

1. Use the style method on our dataframe to get things started
2. Use set_caption to set the title for our styled dataframe
3. Hide the index for our final output
4. Apply the function to add Team colors by row
5. Set the color of the text in the dataframe to white so that it can contrast against the different team colors

In [18]:
from IPython.core.display import display_html

nfc_table = (nfc_wins.style
.hide(axis='index')
.set_caption('NFC Teams')
.set_table_attributes("style='display:inline'")
.apply(highlight_cols, coldict=COLORS, conf='NFC')
.set_properties(**{'color':'white'})
)

afc_table = (afc_wins.style
.hide(axis='index')
.set_caption('AFC Teams')
.set_table_attributes("style='display:inline'")
.apply(highlight_cols, coldict=COLORS, conf='AFC')
.set_properties(**{'color':'white'})
)

display_html(afc_table._repr_html_() + nfc_table._repr_html_(), raw=True)

Team,Projected Wins,Actual Wins,Win Difference
KC,12.00,14,-2.00
TEN,11.41,12,-0.59
CIN,10.54,13,-2.46
BUF,10.45,12,-1.55
LV,9.60,10,-0.40
NE,9.31,10,-0.69
LAC,9.00,9,0.00
IND,9.00,9,0.00
MIA,9.00,9,0.00
PIT,8.62,9,-0.38


In [19]:
projected_wins.sort_values(['Win_Diff'])

,Team,Points For,Points Against,Projected_Wins,Current_Wins,Total_Projected_Wins,Win_Diff
6,LA,567,447,-2.58,16,13.42,-2.58
17,CIN,552,458,-2.46,13,10.54,-2.46
10,KC,588,448,-2.00,14,12.00,-2.00
30,SF,480,412,-1.78,12,10.22,-1.78
0,BUF,566,348,-1.55,12,10.45,-1.55
29,TB,569,398,-1.43,14,12.57,-1.43
15,DAL,547,381,-0.72,12,11.28,-0.72
14,NE,479,350,-0.69,10,9.31,-0.69
12,GB,460,384,-0.61,13,12.39,-0.61
26,TEN,435,373,-0.59,12,11.41,-0.59
